# Oracle Eloqua Testing

### Setup

In [1]:
%autosave 600

import os
import sys

Autosaving every 600 seconds


In [2]:
# stupid and lazy way to set up location; 
# doesn't matter since this file will go bye-bye
proxies = {}
if os.name == 'nt':
    prefix = 'H:'
    proxies['http'] = proxies['https'] = 'wsa04proxy.int.capella.lan:80'
else:
    prefix = '/Users/nickolasgryga'

sys.path.append(prefix + '/work/oracle_eloqua')
creds_loc = prefix + '/work/adhoc/credentials.json'

### Test Import

In [3]:
import oracle_eloqua

### Test API Connection

In [4]:
from oracle_eloqua import EloquaApi
api = oracle_eloqua.EloquaApi.from_json(creds_loc, proxies=proxies)
print(api)

### Test Cursor

In [5]:
from oracle_eloqua import Cursor
request = Cursor(
    path=('assets','campaigns'),
    api_type='REST',
    params={'search':'currentStatus=7'},
    api=api
)

In [6]:
request

{}

In [7]:
response = request.execute()

In [8]:
print(len(response['elements']))
print(response['total'])

7843
7843


### Test Requests

In [9]:
from oracle_eloqua import EloquaRequest
request = EloquaRequest(
    method='GET',
    endpoint='/assets/campaigns',
    api_type='REST'
)

In [10]:
response = request.execute()

In [11]:
print(len(response['elements']))
print(response['total'])

10779
10779


### Test Objects

In [12]:
from oracle_eloqua import EloquaRequest
request = EloquaRequest(
    method='GET',
    endpoint='/assets/campaign',
    obj_id=23,
    api_type='REST'
)

In [13]:
response = request.execute()

In [14]:
from oracle_eloqua.objects import EloquaObject
campaign = EloquaObject.create_object(data=response)

In [15]:
all(hasattr(campaign, attr) for attr in response.keys())

True

### Test BULK API

In [16]:
from oracle_eloqua import EloquaRequest
request = EloquaRequest(
    method='POST',
    api_type='BULK',
    endpoint='/contacts/exports'    
)
json = {
    'name': 'test',
    'fields': {
        "registration_date": "{{Contact.Field(C_Registration_Date1)}}",
        "application_date": "{{Contact.Field(C_Application_Date1)}}",
        "email_address": "{{Contact.Field(C_EmailAddress)}}"
    },
    'filter': "'{{Contact.Field(C_Application_Date1)}}' >= '2017-12-30'"
}
request.add_params(json)
response = request.execute()
print(response)

{'name': 'test', 'fields': {'registration_date': '{{Contact.Field(C_Registration_Date1)}}', 'application_date': '{{Contact.Field(C_Application_Date1)}}', 'email_address': '{{Contact.Field(C_EmailAddress)}}'}, 'filter': "'{{Contact.Field(C_Application_Date1)}}' >= '2017-12-30'", 'dataRetentionDuration': 'PT12H', 'uri': '/contacts/exports/1080260', 'createdBy': 'Nick.Gryga', 'createdAt': '2019-02-01T19:58:02.4636652Z', 'updatedBy': 'Nick.Gryga', 'updatedAt': '2019-02-01T19:58:02.4636652Z'}


In [17]:
request = EloquaRequest(
    method='POST',
    api_type='BULK',
    endpoint='/syncs'    
)

request.add_params({'syncedInstanceUri': '/contacts/exports/1080215'})
response = request.execute()
print(response)

{'syncedInstanceUri': '/contacts/exports/1080215', 'status': 'pending', 'createdAt': '2019-02-01T19:58:02.8697592Z', 'createdBy': 'Nick.Gryga', 'uri': '/syncs/2159669'}


In [18]:
from oracle_eloqua import EloquaRequest
request = EloquaRequest(
    method='GET',
    api_type='BULK',
    endpoint='/syncs',
    obj_id = 2159592
)
response = request.execute()
print(response)

{'syncedInstanceUri': '/contacts/exports/1080215', 'syncStartedAt': '2019-02-01T19:42:33.9930000Z', 'syncEndedAt': '2019-02-01T19:42:34.6300000Z', 'status': 'success', 'createdAt': '2019-02-01T19:42:33.8730000Z', 'createdBy': 'Nick.Gryga', 'uri': '/syncs/2159592'}


In [5]:
from oracle_eloqua import EloquaRequest
request = EloquaRequest(
    method='GET',
    api_type='BULK',
    endpoint='/syncs/2159592/data',
)
response = request.execute()

In [6]:
list(response)

['items', 'totalResults', 'hasMore']

In [7]:
len(response['items'])

50000